Setup code

In [49]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [63]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogeneity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Mahjong game?
"""

In [64]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
You are the AI that separates instruction and context from the user input.
In this query, separate the instruction and context.
Do not answer to the query, just separate it. Also, only use given query itself, do not generate or change any words.
Instruction is saying about what AI should do.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided(N/A for this case).
Instruction and context must not have duplicate contents.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": query2})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

Prove or disprove the statement that the linear transformation of the subspace is also a subspace based on the given contents.
Linear transformation is a function that satisfies two features. One is homogeneity, and the other is additivity.


In [65]:
# 각 chain을 실행하고 결과 누적으로 얻음
# (전체 히스토리 누적 후) 최종 결과 질문

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", give me what to do to achieve this instruction for AI.
Only print 1., 2., 3., ... steps to do. Do not answer to the question, and only print steps.
Each step should be a one-liner.
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context}).split("\n")
print(answer)
print(len(answer))

['1. Understand the definition of a subspace: A subspace of a vector space is a subset of the vector space that is closed under vector addition and scalar multiplication.', '2. Understand the definition of a linear transformation: A linear transformation is a function that preserves the operations of vector addition and scalar multiplication.', '3. Apply the definition of a subspace to the image of the linear transformation: The image of a linear transformation is the set of all output vectors for all input vectors.', '4. Prove or disprove whether the image of the linear transformation is closed under vector addition and scalar multiplication.', '5. If the image is closed under vector addition and scalar multiplication, then the image is a subspace.', '6. If the image is not closed under vector addition or scalar multiplication, then the image is not a subspace.']


In [66]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
    ]
)

chain = rag_with_history_prompt | llm | StrOutputParser()

history = []

for i in range(len(answer)):
    if (i == 0):
        history.append(
            HumanMessage(answer[i] + "\n Refer to or use this context.\n Context:" + context)
        )
    else:
        history.append(
            HumanMessage(answer[i]) 
        )
    chain_result = chain.invoke({"history": history})
    history.append(
        AIMessage(chain_result)
    )

history.append(
    HumanMessage(instr)
)
answer = chain_result = chain.invoke({"history": history})
print(answer)


The statement that the linear transformation of a subspace is also a subspace can be proven based on the given contents. A subspace of a vector space is defined as a subset of the vector space that is closed under vector addition and scalar multiplication.

A linear transformation is a function that maps vectors from one vector space to another, preserving the operations of vector addition and scalar multiplication. This means that for any two vectors u and v in the domain vector space, and any scalar c, the following properties hold:

1. Homogeneity: T(c * u) = c * T(u)
2. Additivity: T(u + v) = T(u) + T(v)

Given that the domain of the linear transformation is a subspace, and the transformation preserves the operations of vector addition and scalar multiplication, it follows that the image of the transformation, which is the range of all possible output vectors, will also be closed under these operations.

Therefore, the image of a linear transformation, when the domain is a subspace